# Midterm

## Options

In [1]:
train_ModelA = False
train_SRGAN = True
gen_dataset = True
train_ModelB = True

## Imports

In [2]:
import numpy as np
from tensorflow import keras
import tensorflow as tf
import os
import datetime
import shutil
import re
import time

logs_base_dir = "./logs"
shutil.rmtree(logs_base_dir, ignore_errors=True)
os.makedirs(logs_base_dir, exist_ok=True)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu,True)
    except RuntimeError as e:
        print(e)

%load_ext tensorboard

2023-10-31 18:50:17.908433: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-31 18:50:17.941309: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-31 18:50:18.642752: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-31 18:50:19.649931: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-31 1

## Step 1: Training a binary classifier

### Separating the data

In [3]:
label_list = []
cats = [0 for i in range(12500)]
dogs = [1 for i in range(12500)]
label_list.extend(cats)
label_list.extend(dogs)

In [4]:

ds = keras.utils.image_dataset_from_directory(
    directory='data/dogs_cats/train/',
    labels=label_list,
    label_mode='categorical',
    batch_size=64,
    image_size=(128,128),
    seed=0,
    validation_split=0.1,
    subset="both"
)
(train_ds, test_ds) = ds

Found 25000 files belonging to 2 classes.
Using 22500 files for training.
Using 2500 files for validation.


2023-10-31 18:53:48.213317: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-31 18:53:48.213414: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-31 18:53:48.213433: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-31 18:53:48.794315: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-31 18:53:48.794387: I tensorflow/compile

### Preprocess and Augment the data

In [5]:
data_augmentation = keras.Sequential(
    [
        keras.layers.RandomFlip("horizontal"),
        keras.layers.RandomRotation(0.1),
        keras.layers.RandomFlip("vertical"),
        keras.layers.RandomZoom(height_factor=0.1,width_factor=0.1)
    ]
)

### Perform Transfer Learning

### MobileNetV2

In [6]:
base_mobile_model = keras.applications.MobileNetV2(
    weights="imagenet",
    input_shape=(128,128,3),
    include_top=False
)

base_mobile_model.trainable = False

inputs = keras.Input(shape=(128,128,3))
x = data_augmentation(inputs)

scaled_layer = keras.layers.Rescaling(scale=1/255.0)
x = scaled_layer(x)

x = base_mobile_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(2)(x)
mobile_model = keras.Model(inputs, outputs)

mobile_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 128, 128, 3)       0         
                                                                 
 rescaling (Rescaling)       (None, 128, 128, 3)       0         
                                                                 
 mobilenetv2_1.00_128 (Func  (None, 4, 4, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 1280)              0     

In [7]:
if train_ModelA:
    keras.backend.clear_session()
    mobile_model.compile(
        optimizer=keras.optimizers.Adam(),
        loss=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.CategoricalAccuracy()],
    )

    logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

    epochs = 20
    mobile_model.fit(train_ds, epochs=epochs, callbacks=[tensorboard_callback])

#### Fine Tuning

In [8]:
if train_ModelA:
    keras.backend.clear_session()
    base_mobile_model.trainable = True

    mobile_model.compile(
        optimizer=keras.optimizers.Adam(1e-5),
        loss=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.CategoricalAccuracy()],
    )

    mobile_model.fit(train_ds, epochs=10, callbacks=[tensorboard_callback])

### Saving Model A

In [9]:
if train_ModelA:
    mobile_model.save_weights("models/modelA.ckpt")
else:
    mobile_model.load_weights("models/modelA.ckpt")

## Step 2: Training SRGAN

In [10]:
def GENBlock(x):
    out = keras.layers.Conv2D(64, (3,3), (1,1), padding='same')(x)
    out = keras.layers.BatchNormalization()(out)
    out = keras.layers.PReLU()(out)
    out = keras.layers.Conv2D(64, (3,3), (1,1), padding='same')(x)
    out = keras.layers.BatchNormalization()(out)
    out = keras.layers.Add()([out, x])
    return out

def DISBlock(x, n):
    out = keras.layers.Conv2D(n, (3,3), (1,1), padding='same')(x)
    out = keras.layers.BatchNormalization()(out)
    out = keras.layers.LeakyReLU(0.2)(out)
    out = keras.layers.Conv2D(n, (3,3), (2,2), padding='same')(x)
    out = keras.layers.BatchNormalization()(out)
    out = keras.layers.LeakyReLU(0.2)(out)
    return out


### Generator

In [11]:
inputs = keras.Input(shape=(32,32,3))
scaled_layer = keras.layers.Rescaling(scale=1/127.5, offset=-1)

x = scaled_layer(inputs)
x = keras.layers.Conv2D(64, (9,9), (1,1), padding='same')(x)
x = keras.layers.PReLU()(x)
x_skip = x
for i in range(5):
    x = GENBlock(x)
x = keras.layers.Conv2D(64, (3,3), (1,1), padding='same')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Add()([x, x_skip])

x = keras.layers.Conv2D(256, (3,3), (1,1), padding='same')(x)
x = keras.layers.Reshape((x.shape[1]*2, x.shape[2]*2, x.shape[3]//4))(x)
x = keras.layers.PReLU()(x)

x = keras.layers.Conv2D(256, (3,3), (1,1), padding='same')(x)
x = keras.layers.Reshape((x.shape[1]*2, x.shape[2]*2, x.shape[3]//4))(x)
x = keras.layers.PReLU()(x)

outputs = keras.layers.Conv2D(3, (9,9), (1,1), padding='same')(x)

generator_model = keras.Model(inputs, outputs)
generator_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 rescaling_1 (Rescaling)     (None, 32, 32, 3)            0         ['input_3[0][0]']             
                                                                                                  
 conv2d (Conv2D)             (None, 32, 32, 64)           15616     ['rescaling_1[0][0]']         
                                                                                                  
 p_re_lu (PReLU)             (None, 32, 32, 64)           65536     ['conv2d[0][0]']              
                                                                                            

### Discriminator

In [12]:
inputs = keras.Input(shape=(128,128,3))
scaled_layer = keras.layers.Rescaling(scale=1/127.5, offset=-1)

x = scaled_layer(inputs)
x = keras.layers.Conv2D(64, (3,3), (1,1), padding='same')(x)
x = keras.layers.LeakyReLU(0.2)(x)
x = keras.layers.Conv2D(64, (3,3), (2,2), padding='same')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.LeakyReLU(0.2)(x)
x = DISBlock(x, 128)
x = DISBlock(x, 256)
x = DISBlock(x, 512)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(1024)(x)
x = keras.layers.LeakyReLU(0.2)(x)
x = keras.layers.Dense(1)(x)

# outputs = keras.activations.sigmoid(x)
outputs = x

discriminator_model = keras.Model(inputs, outputs)
discriminator_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 rescaling_2 (Rescaling)     (None, 128, 128, 3)       0         
                                                                 
 conv2d_15 (Conv2D)          (None, 128, 128, 64)      1792      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 128, 128, 64)      0         
                                                                 
 conv2d_16 (Conv2D)          (None, 64, 64, 64)        36928     
                                                                 
 batch_normalization_11 (Ba  (None, 64, 64, 64)        256       
 tchNormalization)                                               
                                                           

### Training

In [13]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [14]:
if gen_dataset:
    ds_small = keras.utils.image_dataset_from_directory(
        directory='data/dogs_cats/train/',
        labels=label_list,
        label_mode='categorical',
        batch_size=64,
        image_size=(32,32),
        seed=0,
        validation_split=0.1,
        subset="both"
    )
    (train_ds_small, test_ds_small) = ds_small

Found 25000 files belonging to 2 classes.
Using 22500 files for training.
Using 2500 files for validation.


In [15]:
@tf.function
def train_step(small_image, big_image):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator_model(small_image, training=True)

        real_output = discriminator_model(big_image, training=True)
        fake_output = discriminator_model(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

        grad_gen = gen_tape.gradient(gen_loss, generator_model.trainable_variables)
        grad_disc = disc_tape.gradient(disc_loss, discriminator_model.trainable_variables)

        generator_optimizer.apply_gradients(zip(grad_gen, generator_model.trainable_variables))
        discriminator_optimizer.apply_gradients(zip(grad_disc, discriminator_model.trainable_variables))

def train(small_dataset, big_dataset, epochs):
    for epoch in range(epochs):
        start = time.time()
        iterator = iter(big_dataset)
        for small_batch in small_dataset:
            big_batch = iterator.get_next()
            train_step(small_batch[0], big_batch[0])

        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

In [16]:
if train_SRGAN:
    train(train_ds_small, train_ds, 50)

2023-10-31 18:57:21.789992: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8801
2023-10-31 18:57:23.581456: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-31 18:57:27.822951: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fd7cc56c380 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-31 18:57:27.822998: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2023-10-31 18:57:27.828131: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-31 18:57:27.909982: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the p

Time for epoch 1 is 69.91319370269775 sec
Time for epoch 2 is 51.95230412483215 sec
Time for epoch 3 is 51.697054624557495 sec


KeyboardInterrupt: 

In [ ]:
if train_SRGAN:
    generator_model.save_weights("models/gen_model.ckpt")
    discriminator_model.save_weights("models/dis_model.ckpt")
else:
    generator_model.load_weights("models/gen_model.ckpt")
    discriminator_model.load_weights("models/dis_model.ckpt")